In [ ]:
%%pyspark 

from pyspark.sql.functions import explode
from pyspark.sql.functions import col, concat_ws


filewqual = "esgupload/*"

### you need to change the path to point the containing folder of the esgupload folder. this folder must contain only
###  files pushed from the power automate ESG extract flow
thefile = "abfss://workspace@stgbvnews.dfs.core.windows.net/" + filewqual + ".json"
df_raw = spark.read.load(thefile, format='json')

df_raw.printSchema()

### flatten the phrases and referenced orgs


ro = df_raw.select(df_raw.title, df_raw.excerpt, df_raw.phrases, df_raw.themes, df_raw.sentiment, df_raw.significance, df_raw.signals,  explode(df_raw.referencedOrgs).alias("refdOrg"))\
                .withColumn("refdOrg", col("refdOrg.name"))\
                .withColumn("phrases", concat_ws(",", col("phrases")))

ro.printSchema()




### explode the signals and themes
signals_ex = ro.select(ro.title,ro.excerpt, ro.phrases, ro.themes, ro.sentiment, ro.significance, ro.refdOrg, explode(ro.signals).alias("signal"))
df_final  = signals_ex.select(signals_ex.signal,
    signals_ex.title,signals_ex.excerpt, signals_ex.phrases,  signals_ex.sentiment, signals_ex.significance, signals_ex.refdOrg, explode(signals_ex.themes).alias("theme"))



spark.sql("CREATE DATABASE IF NOT EXISTS bitvorenews")


### overwrite the  table
df_final.write.mode("overwrite").saveAsTable("bitvorenews.sigtheme_ex")

df_final.printSchema()
display(df_final.limit(100))

In [ ]:
%%spark
val sql_write_df = spark.sql("SELECT title, significance, refdOrg, signal, theme, phrases, sentiment FROM bitvorenews.sigtheme_ex ")

sql_write_df.write.mode("overwrite").sqlanalytics("bitvoresql.dbo.sigtheme_all", Constants.INTERNAL)